<a href="https://colab.research.google.com/github/Melud/BNP_project/blob/main/Copie_de_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Unidecode

     |████████████████████████████████| 245kB 5.4MB/s 


In [ ]:
!pip install contractions

     |████████████████████████████████| 327kB 5.4MB/s 
     |████████████████████████████████| 266kB 7.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85399 sha256=7482ccf3a064a6e1d85dcf9c762e2e4f8f638d316afa0b07784a47b3b7c581d2
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [14]:
import pandas as pd
import numpy as np
import re
import string 
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/NLP project

/content/drive/MyDrive/NLP project


# DATA

In [5]:
train = pd.read_csv('train.csv', sep=',')
print(train.shape)

(159571, 8)


In [6]:
train.head(15)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [7]:
tokenizer = get_tokenizer('basic_english')



def clean_text(text):
  text=text.lower()
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  text=url_pattern.sub(r'', text)
  text=re.sub(r'\d+', '',text)
  translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  text=text.translate(translator)
  text=re.sub(r'\n',' ',text)
  text= re.sub(' +', ' ', text)
  return(text.strip())


  

In [8]:
X_train=train["comment_text"].apply(clean_text)
X_train.head()

0    explanation why the edits made under my userna...
1    d aww he matches this background colour i m se...
2    hey man i m really not trying to edit war it s...
3    more i can t make any real suggestions on impr...
4    you sir are my hero any chance you remember wh...
Name: comment_text, dtype: object

In [57]:
y_train=train[train.columns[2:]].apply(lambda x:np.array(list(x)),axis=1)
print(y_train)
print(len(y_train))
num_class=y_train[0].size
print(num_class)

0         [0, 0, 0, 0, 0, 0]
1         [0, 0, 0, 0, 0, 0]
2         [0, 0, 0, 0, 0, 0]
3         [0, 0, 0, 0, 0, 0]
4         [0, 0, 0, 0, 0, 0]
                 ...        
159566    [0, 0, 0, 0, 0, 0]
159567    [0, 0, 0, 0, 0, 0]
159568    [0, 0, 0, 0, 0, 0]
159569    [0, 0, 0, 0, 0, 0]
159570    [0, 0, 0, 0, 0, 0]
Length: 159571, dtype: object
159571
6


In [10]:
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for text in X_train:
  counter.update(tokenizer(text))
vocab = Vocab(counter, min_freq=3)



In [56]:
vocab_size=len(vocab)
print(vocab_size)

59868


In [11]:
max_len=X_train.map(lambda x:len(tokenizer(x))).max()
print(max_len)

1403


In [12]:
vocab['<pad>']

1

In [37]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [140]:
class ToxicCommentDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, X,y):

        self.X =X
        self.y =y
        

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
       
        return (self.X[idx],self.y[idx])


def collate_batch(batch):
    vectorized_seqs=[]
    list_label=[]
    for text , label in batch:
      vectorized_seqs.append(text_pipeline(text))
      list_label.append(label)
      
    seq_lengths=list(map(len, vectorized_seqs))
    seq_tensor=torch.ones(size=(len(batch),max_len),dtype=torch.int64)
    for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
      seq_tensor[idx, :seqlen] = torch.tensor(seq,dtype=torch.int64)
    
        
    return seq_tensor,torch.tensor(list_label,dtype=torch.int64)


In [165]:
from torch import nn
from torch import autograd
from torch.nn import functional as F
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim,hidden_dim, num_class,batch_size):
        super(TextClassificationModel, self).__init__()
        self.batch_size=batch_size
        self.hidden_dim=hidden_dim

        self.embedding = nn.Embedding( vocab_size,embed_dim, padding_idx=1)
        self.lstm = nn.LSTM(input_size=embed_dim,
                            hidden_size=hidden_dim,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        #self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*hidden_dim,num_class)
       # self.hidden = self.init_hidden()

    '''def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers * num_directions, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(2, self.batch_size, self.hidden_dim)),   
                autograd.Variable(torch.zeros(2, self.batch_size, self.hidden_dim))) '''

    def forward(self, text):
        embed_text= self.embedding(text)
        lstm_out,(hn,cn) = self.lstm(embed_text)
        x=torch.cat((hn[0,:,:],hn[1,:,:]),1)
        return torch.sigmoid(self.fc(x))

      
        
      

        


In [166]:
model=TextClassificationModel(vocab_size,100,50,num_class,8)

In [169]:
toxic_train=ToxicCommentDataset(X_train,y_train)
dataloader = DataLoader(toxic_train, batch_size=8, shuffle=False,collate_fn=collate_batch)

for i,(u,l) in enumerate(dataloader):
  
  print(model(u))
 
  if i==0:
    break
   

tensor([[0.4956, 0.5503, 0.4635, 0.4638, 0.4869, 0.4973],
        [0.4972, 0.5113, 0.4568, 0.4486, 0.5102, 0.4949],
        [0.5134, 0.4677, 0.4557, 0.4867, 0.5060, 0.5042],
        [0.4980, 0.4831, 0.4598, 0.4923, 0.5015, 0.5103],
        [0.4675, 0.5204, 0.4753, 0.4779, 0.4990, 0.4994],
        [0.4814, 0.5221, 0.4807, 0.4739, 0.5220, 0.5184],
        [0.4689, 0.5158, 0.4696, 0.4800, 0.4931, 0.4971],
        [0.4742, 0.5212, 0.4645, 0.4822, 0.5276, 0.5036]],
       grad_fn=<SigmoidBackward>)


In [148]:
x=np.arange(40).reshape(2,2,10)
print(x.shape)
print(x[0,:,:],x[1,:,:])
print(np.concatenate((x[0,:,:],x[1,:,:]),axis=1).shape)

(2, 2, 10)
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]] [[20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]]
(2, 20)
